In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="MIG-ff70592b-6c77-5bde-832d-88d1e18cad50" 
from hydra.utils import instantiate
import yaml
import torch
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from PIL import Image
device = "cuda:0"

# Load data and models

In [2]:
os.chdir('/allen/aics/modeling/ritvik/projects/benchmarking_representations/')
save_path = './test_var_punctate_embeddings/'

In [3]:
from br.data.get_datamodules import get_data
from cyto_dl.models.utils.mlflow import load_model_from_checkpoint
from br.models.load_models import load_model_from_path
from br.models.save_embeddings import get_pc_loss

def get_data_and_models(dataset_name, batch_size, results_path, debug=False):
    data_list = get_data(dataset_name, batch_size, results_path, debug)
    all_models, run_names, model_sizes = load_model_from_path(dataset_name, results_path) # default list of models in load_models.py
    return data_list, all_models, run_names, model_sizes

In [6]:
run_names

['classical_image',
 'so3_image',
 '2048_ed_dgcnn',
 '2048_int_ed_vndgcnn',
 '2048_int_ed_vndgcnn_structnorm']

In [10]:
dataset_name = 'other_punctate'
batch_size = 2
debug=False
results_path = '/allen/aics/modeling/ritvik/projects/benchmarking_representations/configs/results/'
data_list, all_models, run_names, model_sizes = get_data_and_models(dataset_name, batch_size, results_path, debug)

In [11]:
data_list[0]

# compute embeddings and emissions

In [ ]:
from br.models.save_embeddings import save_embeddings

splits_list = ["train", 'val', "test"]
meta_key = None
eval_scaled_img = [False] * 5
eval_scaled_img_params = [{}] * 5
loss_eval_list = None
sample_points_list = [True, True, False, False, False]
skew_scale = 100
save_embeddings(
    save_path,
    data_list,
    all_models,
    run_names,
    debug,
    splits_list,
    device,
    meta_key,
    loss_eval_list,
    sample_points_list,
    skew_scale,
    eval_scaled_img,
    eval_scaled_img_params,
)

Processing train


 49%|███████████████████████████████████████████████████▍                                                      | 12174/25085 [35:00<39:48,  5.41it/s]Exception ignored in: <function _ConnectionBase.__del__ at 0x7fbab610ab00>
Traceback (most recent call last):
  File "/home/ritvik.vasan/anaconda3/envs/replearn/lib/python3.10/multiprocessing/connection.py", line 132, in __del__
    self._close()
  File "/home/ritvik.vasan/anaconda3/envs/replearn/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Traceback (most recent call last):
  File "/home/ritvik.vasan/anaconda3/envs/replearn/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/home/ritvik.vasan/anaconda3/envs/replearn/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/home/ritvik.vasan/anaconda3/envs/replearn/lib/python3.10/multiprocessing/connection.py", line 361, in _close
  

Processing val


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5376/5376 [15:50<00:00,  5.66it/s]


Processing test


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5376/5376 [15:47<00:00,  5.68it/s]


Processing train


 48%|█████████████████████████████████████████████████▍                                                    | 12143/25085 [1:03:37<1:07:09,  3.21it/s]